In [4]:
import sys
sys.path.append('../')

from typing import List, Dict

from src.config.project_paths import  get_data_file_path, \
     get_voxceleb2_metadata_path, get_voxceleb2_path, create_wav_dir_if_not_exists, get_project_root_path, get_voxceleb2_wav_path
import pandas as pd

## VoxCeleb 2

In [5]:
voxceleb2_metadata = pd.read_csv(get_voxceleb2_metadata_path(), sep=',')
voxceleb2_metadata = voxceleb2_metadata.drop(columns=["VGGFace2 ID "], axis=1).rename(
    columns={"VoxCeleb2 ID ": "user_id", "Gender ": "gender", "Set ": "set"})
voxceleb2_metadata["user_id"] = voxceleb2_metadata["user_id"].str.strip()
voxceleb2_metadata["gender"] = voxceleb2_metadata["gender"].str.strip()
voxceleb2_metadata["set"] = voxceleb2_metadata["set"].str.strip()

voxceleb2_metadata.head()

,user_id,gender,set
0,id00012,m,dev
1,id00015,m,dev
2,id00016,m,dev
3,id00017,m,test
4,id00018,m,dev


In [8]:
voxceleb2_path = get_voxceleb2_wav_path()
voxceleb2_path

PosixPath('/Users/jjaniak/Documents/studia/biometria/projekt2/VoiceAuthenticator/data/vox2_test/wav')

In [9]:
id_paths = voxceleb2_path.iterdir()

<generator object Path.iterdir at 0x168616b20>

In [13]:
import random

id_paths_list = list(voxceleb2_path.iterdir())
random_ids = random.sample(id_paths_list, 40)


In [14]:
annotations: List[Dict[str, any]] = []

In [23]:
import os

wav_file_path = get_voxceleb2_wav_path()

for id_path in random_ids:
    if id_path.stem not in [id_path.stem for id_path in random_ids]:
        continue
    create_wav_dir_if_not_exists(id_path.name)
    sample_paths = list(id_path.iterdir())
    
    for sample_path in sample_paths[:5]:
        wav_dir = os.path.join(get_voxceleb2_wav_path(), f"{id_path.name}/{sample_path.name}")
        os.makedirs(wav_dir, exist_ok=True)
        wav_paths = sample_path.iterdir()
        for wav_path in wav_paths:
            annotations.append(
                {
                    "wav_path": f"/data/vox2_test/wav/{id_path.name}/{sample_path.name}/{wav_path.stem}.wav",
                    "user_id": id_path.stem
                }
            )
            

In [24]:
len(annotations)

1341

In [25]:
voxceleb2_user_to_metadata = voxceleb2_metadata[["user_id", "gender"]].drop_duplicates().set_index("user_id").to_dict(orient="index")

In [26]:
for annotation in annotations:
    annotation["gender"] = voxceleb2_user_to_metadata[annotation["user_id"]]["gender"]

In [29]:
pd.DataFrame(annotations).to_json(get_data_file_path("annotations_original.json"), orient="records", default_handler=str)